## Next location prediction

The user inputs current location and city.

Based on pretrained LSTM model, the code will predict based on frequent POI movements the next location for the user to go to

In [1]:
# import libraries
import numpy as np
import json
from tensorflow.keras.models import load_model


In [8]:
# Load LSTM models for all cities
models = {
    'A': load_model('weights\lstm_city_A.h5'),
    'B': load_model('weights\lstm_city_B.h5'),
    'C': load_model('weights\lstm_city_C.h5'),
    'D': load_model('weights\lstm_city_D.h5'),
}


In [9]:
# Load POI mappings
poi_mappings = {}
for city in ['A', 'B', 'C', 'D']:
    with open(f'poi_mapping\poi_mapping_city_{city}.json', 'r') as f:
        poi_mapping_json_compatible = json.load(f)
        # Convert string keys back to tuples
        poi_mappings[city] = {eval(key): value for key, value in poi_mapping_json_compatible.items()}


In [ ]:
def predict_next_location(city, current_location):
    # Validate inputs
    if city not in models:
        return {'error': 'Invalid city'}
    if not isinstance(current_location, list) or len(current_location) != 2:
        return {'error': 'Invalid current location'}

    # Load the corresponding model
    model = models[city]
    poi_mapping = poi_mappings[city]

    # Generate a padded sequence
    sequence_length = 10
    feature_size = model.input_shape[-1]
    sequence = np.zeros((sequence_length, feature_size))
    sequence[-1, :2] = current_location  
    
    # Reshape input for LSTM
    sequence = np.expand_dims(sequence, axis=0)

    # Predict the next location
    prediction = model.predict(sequence)
    predicted_location = [round(coord) for coord in prediction[0]] 

    # Find the nearest POI grid
    poi_grids = list(poi_mapping.keys())  
    distances = [np.sqrt((predicted_location[0] - grid[0])**2 + (predicted_location[1] - grid[1])**2) for grid in poi_grids]
    nearest_poi_index = np.argmin(distances)
    nearest_poi_grid = poi_grids[nearest_poi_index]

    # Get the POI for the nearest grid
    poi = poi_mapping.get(nearest_poi_grid, "Unknown")

    # Return the results
    return {
        'predicted_location': predicted_location,
        'nearest_poi_location': list(nearest_poi_grid),
        'poi': poi
    }


In [11]:
# Example input
city = 'D'
current_location = [10, 15]

# Run prediction
result = predict_next_location(city, current_location)

# Display the result
print("Predicted Results:")
print(f"Predicted Next Location (x, y): {result['predicted_location']}")
print(f"Nearest POI Location (x, y): {result['nearest_poi_location']}")
print(f"POI: {result['poi']}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
Predicted Results:
Predicted Next Location (x, y): [53, 46]
Nearest POI Location (x, y): [54, 46]
POI: Hair Salon
